In [1]:
%pip install voyageai numpy

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 16.7/16.7 MB 30.5 MB/s  0:00:00 eta 0:00:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 11/11 [voyageai]/11 [aiohttp]t]
Note: you may need to restart the kernel to use updated packages.


In [2]:
import os
import json
import voyageai
import numpy as np
from dotenv import load_dotenv

load_dotenv()

voyage = voyageai.Client(api_key=os.getenv("VOYAGE_API_KEY"))

# ─── 1. Convertir une station en texte lisible (le "chunk") ───

def station_to_text(station: dict) -> str:
    return f"""
Station : {station['name']} ({station['region']})
Altitude : {station['altitude_min']}m - {station['altitude_max']}m
Domaine : {station['ski_area']} — {station['km_slopes']} km de pistes
Pistes : {station['slopes_detail']['green']} vertes, {station['slopes_detail']['blue']} bleues,
         {station['slopes_detail']['red']} rouges, {station['slopes_detail']['black']} noires
Niveaux : {', '.join(station['level'])}
Forfait journée adulte : {station['passes']['full_day']['adult']}€
Hébergement moyen : {station['avg_accommodation_price']}€/nuit
Activités : {', '.join(station['activities'])}
Services : {', '.join(station['services'])}
Description : {station['description']}
Aéroport le plus proche : {station['access']['nearest_airport']} ({station['access']['distance_from_airport_km']} km)
""".strip()

# Charger les stations
with open("../data/stations/stations.json", "r") as f:
    stations = json.load(f)

# Visualiser un chunk
les_orres = next(s for s in stations if s['id'] == 'les-orres')
print(station_to_text(les_orres))

Station : Les Orres (Hautes-Alpes)
Altitude : 1500m - 2500m
Domaine : Les Orres — 100 km de pistes
Pistes : 15 vertes, 30 bleues,
         35 rouges, 20 noires
Niveaux : beginner, intermediate, advanced
Forfait journée adulte : 39€
Hébergement moyen : 60€/nuit
Activités : snowshoeing, snow_scooter
Services : ski_school, equipment_rental
Description : Ski dans les Alpes du Sud, ambiance conviviale
Aéroport le plus proche : Marseille (230 km)


In [3]:
# ─── 2. Créer un embedding et l'explorer ───

chunk = station_to_text(les_orres)
result = voyage.embed([chunk], model="voyage-3")
embedding = result.embeddings[0]

print(f"Type    : {type(embedding)}")
print(f"Taille  : {len(embedding)} dimensions")  # → 1024
print(f"Extrait : {embedding[:5]}")              # → liste de floats

Type    : <class 'list'>
Taille  : 1024 dimensions
Extrait : [-0.03828684240579605, -0.02504226751625538, -0.05431389436125755, -0.052310511469841, -0.03984503075480461]


In [4]:
# ─── 3. La vraie magie : comparer deux stations ───
# La similarité cosinus mesure si deux vecteurs "pointent dans la même direction"
# → 1.0 = identiques, 0.0 = sans rapport, -1.0 = opposés

def cosine_similarity(a, b):
    a, b = np.array(a), np.array(b)
    return np.dot(a, b) / (np.linalg.norm(a) * np.linalg.norm(b))

# Embeddings de 3 stations
val_thorens  = next(s for s in stations if s['id'] == 'val-thorens')
les_menuires = next(s for s in stations if s['id'] == 'les-menuires')  # aussi dans les 3 Vallées

results = voyage.embed(
    [station_to_text(val_thorens), 
     station_to_text(les_menuires), 
     station_to_text(les_orres)],
    model="voyage-3"
)
emb_vt, emb_lm, emb_lo = results.embeddings

print("Val Thorens ↔ Les Menuires (même domaine) :", 
      round(cosine_similarity(emb_vt, emb_lm), 4))  # → score élevé
print("Val Thorens ↔ Les Orres (domaines différents) :", 
      round(cosine_similarity(emb_vt, emb_lo), 4))  # → score plus bas

Val Thorens ↔ Les Menuires (même domaine) : 0.828
Val Thorens ↔ Les Orres (domaines différents) : 0.6966


In [6]:
# ─── 4. Recherche sémantique manuelle (avant ChromaDB) ───
# C'est exactement ce que fait un vector store, mais à la main

# Embedder toutes les stations
all_texts = [station_to_text(s) for s in stations]
all_results = voyage.embed(all_texts, model="voyage-3")
all_embeddings = all_results.embeddings

def search(query: str, top_k: int = 3):
    # Embedder la question
    query_result = voyage.embed([query], model="voyage-3", input_type="query")
    query_embedding = query_result.embeddings[0]
    
    # Calculer la similarité avec toutes les stations
    scores = [
        (stations[i]['name'], cosine_similarity(query_embedding, emb))
        for i, emb in enumerate(all_embeddings)
    ]
    
    # Trier et retourner les meilleurs résultats
    scores.sort(key=lambda x: x[1], reverse=True)
    return scores[:top_k]

# Test !
print("🔍 'station pour famille débutante pas chère'")
for name, score in search("station pour famille débutante pas chère"):
    print(f"  {name} → {round(score, 4)}")

print("\n🔍 'ski expert haute altitude glacier'")
for name, score in search("ski expert haute altitude glacier"):
    print(f"  {name} → {round(score, 4)}")

🔍 'station pour famille débutante pas chère'
  Superdévoluy → 0.5284
  Montgenèvre → 0.5084
  Valberg → 0.5044

🔍 'ski expert haute altitude glacier'
  Tignes → 0.5686
  Les 2 Alpes → 0.5252
  L'Alpe d'Huez → 0.4998
